# Projeto de Bases de Dados - Parte 3

### Docente Responsável

Prof. Flávio Martins

### Grupo 10 - BD2L12
<dl>
    <dt>6 horas (33.3%)</dt>
    <dd>ist1102492 Francisco Fonseca</dd>
    <dt>6 horas (33.3%)</dt>
    <dd>ist1102604 Gonçalo Rua</dd>
    <dt>6 horas (33.3%)</dt>
    <dd>ist1102611 João Gouveia</dd>
<dl>

In [ ]:
%load_ext sql
%sql postgresql://db:db@postgres/db

# Empresa de comércio online

## 0. Carregamento da Base de Dados

Carregue o esquema de Base de Dados apresentado no Anexo A.

In [ ]:
%sql -f schema.sql

Crie as instruções para o seu preenchimento de forma consistente, garantindo que todas as consultas SQL e OLAP, apresentadas mais adiante, produzam um resultado não vazio. 

In [ ]:
%sql -f populate.sql

## 1. Restrições de Integridade

Apresente o código para implementar as seguintes restrições de integridade, se necessário, com recurso a extensões procedimentais SQL (Stored Procedures e Triggers):

(RI-1) Nenhum empregado pode ter menos de 18 anos de idade

In [ ]:
%%sql
ALTER TABLE employee
    ADD CHECK (EXTRACT(YEAR FROM (SELECT Now() - bdate)) >= 18);

(RI-2) Um 'Workplace' é obrigatoriamente um 'Office' ou 'Warehouse' mas não pode ser ambos

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION workplace_trigger() RETURNS TRIGGER AS
$$
BEGIN
    IF EXISTS(SELECT address FROM warehouse WHERE address = NEW.address)
        AND EXISTS(SELECT address FROM office WHERE address = NEW.address) THEN
            RAISE EXCEPTION 'Address must be in warehouse or office but not both';
    ELSE
        IF NOT EXISTS (SELECT address FROM office WHERE address = NEW.address) THEN
            RAISE EXCEPTION 'Address must be in warehouse or office';
        END IF;
    END IF;
    RETURN NEW;
END
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_office AFTER INSERT OR DELETE office
    FOR EACH ROW EXECUTE FUNCTION workplace_trigger();
CREATE TRIGGER check_warehouse AFTER INSERT OR DELETE warehouse
    FOR EACH ROW EXECUTE FUNCTION workplace_trigger();
CREATE TRIGGER check_workplace AFTER INSERT workplace
    DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION workplace_trigger();

(RI-3) Uma 'Order' tem de figurar obrigatoriamente em 'Contains'.

In [ ]:
%%sql
CREATE OR REPLACE FUNCTION order_trigger() RETURNS TRIGGER AS
$$
BEGIN
    IF NOT EXISTS(SELECT order_no FROM contains WHERE order_no = NEW.order_no) THEN
        RAISE EXCEPTION 'order_no must exist in contains';
    END IF;
END
$$ LANGUAGE plpgsql;

CREATE TRIGGER check_order AFTER INSERT orders
    DEFERRABLE INITIALLY DEFERRED
    FOR EACH ROW EXECUTE FUNCTION order_trigger();

## 2. Consultas SQL

Apresente a consulta SQL mais sucinta para cada uma das seguintes questões

1) Qual o número e nome do(s) cliente(s) com maior valor total de encomendas pagas?  

In [ ]:
%%sql
SELECT cust_no, name FROM pay
    NATURAL JOIN contains
    NATURAL JOIN product
    NATURAL JOIN customer
    GROUP_BY order_no
    HAVING SUM(price*qty) >= ALL (
        SELECT SUM(price*qty) FROM pay
            NATURAL JOIN constains
            NATURAL JOIN product
            GROUP_BY order_no
    );

2. Qual o nome dos empregados que processaram encomendas em todos os dias de 2022 em que houve encomendas?

In [ ]:
%%sql
SELECT name FROM employee e
    WHERE NOT EXISTS (
        SELECT DISTINCT date FROM orders
            WHERE EXTRACT(YEAR FROM date) = 2022
        EXCEPT
        SELECT DISTINCT date FROM orders
            NATURAL JOIN process
            WHERE ssn = e.ssn AND
                EXTRACT(YEAR FROM date) = 2022
    );

3. Quantas encomendas foram realizadas mas não pagas em cada mês de 2022?

In [ ]:
%%sql
SELECT EXTRACT(MONTH FROM date) AS "month", SUM(*) FROM orders o
    WHERE NOT EXISTS (
        SELECT order_no FROM orders
            WHERE order_no = o.order_no AND
                EXTRACT(YEAR FROM date) = 2022
        EXCEPT
        SELECT order_no FROM pay
            WHERE order_no = o.order_no AND
                EXTRACT(YEAR FROM date) = 2022
    )
    GROUP BY month;

## 3. Vistas

Crie uma vista que resuma as informações mais importantes sobre as vendas de produtos, combinando informações de diferentes tabelas do esquema de base de dados. A vista deve ter o seguinte esquema:

product_sales(sku, order_no, qty, total_price, year, month, day_of_month, day_of_week, city)

In [ ]:
%%sql
CREATE VIEW product_sales AS
    SELECT sku, order_no, qty, SUM(qty*price), EXTRACT(YEAR FROM date), EXTRACT(MONTH FROM date),
        EXTRACT(DAY FROM date), EXTRACT(DOW FROM date), 
            SUBSTRING(address, CHARINDEX('-', address) + 5, 
                LEN(address) - CHARINDEX('-', address) + 6) AS "city" FROM pay
        NATURAL JOIN customer
        NATURAL JOIN orders
        NATURAL JOIN contains
        NATURAL JOIN product
        GROUP BY sku, order_no;

## 4. Desenvolvimento de Aplicação

### Explicação da arquitetura da aplicação web, incluindo um link para uma versão de trabalho e as relações entre os vários ficheiros na pasta web/arquivos

...

## 5. Consultas OLAP

Usando a vista desenvolvida para a Questão 3, escreva duas consultas SQL que permitam analisar:

1. As quantidade e valores totais de venda de cada produto em 2022, globalmente, por cidade, por mês, dia do mês e dia da semana

In [ ]:
%%sql
-- SELECT ...

2. O valor médio diário das vendas de todos os produtos em 2022, globalmente, por mês e dia da semana

In [ ]:
%%sql
-- SELECT ...

## 6. Índices

Indique, com a devida justificação, que tipo de índice(s), sobre qual(is) atributo(s) e sobre qual(is) tabela(s) faria sentido criar, de forma a agilizar a execução de cada uma das seguintes consultas: 

### 6.1
SELECT order_no
FROM orders 
JOIN contains USING (order_no) 
JOIN product USING (SKU) 
WHERE price > 50 AND 
EXTRACT(YEAR FROM date) = 2023

### Tipo de Índice, Atributos & Justificação

Índice B-Tree sobre o atributo price da tabela product para otimizar o filtro **price > 50**, por ser o índice ideal para consultas de range.

Índice Hash sobre o atributo order_no da tabela orders e da tabela contains e sobre o atributo SKU da tabela contains e da tabela product, com o propósito de otimizar a operação JOIN, já que este é o melhor para seleção por igualdade. (?)

Índice Hash sobre o atributo date para otimizar o filtro **date = 2023**, por ser o índice ideal para seleção por igualdade.

### 6.2
SELECT order_no, SUM(qty*price)
FROM contains 
JOIN product USING (SKU) 
WHERE name LIKE ‘A%’ 
GROUP BY order_no;

### Tipo de Índice, Atributos & Justificação

...